<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_data_import_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Import Pipeline

This code is still under construction, and is therefore very very bad in places. 

### TODO
- Import CO2 dataset
- Extract unnecessary methods into normal python files and import where necessary. 
- Remove unnecessary variable changes where necessary - this stacks up all the JSON, making everything harder than it needs to be. 
- Change unnecessary image imports to feature imports 

## Setup
*   Import necessary libraries
*   Set up Earth Engine authentication and mount google drive  


In [1]:
import ee
import folium
import os

from google.colab import drive
from osgeo import gdal
from PIL import Image
from pprint import pprint

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
drive.mount('/content/drive')

print(folium.__version__)

# Dataset import

### Import the following datasets into Google Drive

*   [Sentinel-2 Satellite photography](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
*   [Sentinel-5 Precursor Data](https://developers.google.com/earth-engine/datasets/catalog/sentinel)
  *   [Carbon Monoxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CO)
  *   [Formaldehyde](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_HCHO)
  *   [Nitrogen Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2)
  *   [Ozone](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3)
  *   [Sulphur Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2)
  *   [Methane](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CH4)
*   [ODIAC Fossil Fuel CO2 Emissions](https://db.cger.nies.go.jp/dataset/ODIAC/DL_odiac2019.html)

##### Define Earth Engine Boundaries 

In [4]:
# Define dataset boundaries for britain and london 
great_britain = ee.Geometry.Polygon(
        [[[-1.836112801004015, 59.808076330562756],
          [-8.779472176004015, 58.82140293049428],
          [-7.988456551004015, 55.71069203454839],
          [-11.196464363504015, 54.42753859549109],
          [-11.328300301004015, 50.967746003015044],
          [-9.526542488504015, 50.77361752815123],
          [-6.274589363504015, 51.81776248652293],
          [-5.395683113504015, 51.21615275310099],
          [-6.582206551004015, 49.56332371186494],
          [-3.110526863504015, 49.904165426606255],
          [1.240059073995985, 50.80139967619036],
          [2.426582511495985, 52.33095407387208],
          [1.767402823995985, 53.4183511305661],
          [0.5369340739959849, 53.44453305344514],
          [-1.616386238504015, 56.32474216074427],
          [-0.7814253010040151, 57.805828290000164]]])

london = ee.Geometry.Polygon(
        [[[-0.8034984064310513, 51.86893967293553],
          [-0.8034984064310513, 51.10101596040769],
          [0.4077442693501787, 51.10101596040769],
          [0.4077442693501787, 51.86893967293553]]])

uxbridge = ee.Geometry.Rectangle(
        [[-0.5585304622116949, 51.577993458567235],
         [-0.3951088313523199, 51.51009512268249]])

# Used to check scaling is a-ok
millennium_dome = ee.Geometry.Polygon(
        [[[0, 51.51],
          [0, 51.49],
          [0.02, 51.49],
          [0.02, 51.51]]]);

greenwich = ee.Geometry.Polygon(
        [[[-0.0, 51.55],
          [-0.0, 51.45],
          [0.1, 51.45],
          [0.1, 51.55]]])

west_hemisphere = ee.Geometry.Polygon(
        [[[-177.32831443211026, 84.89714695160266],
          [-177.32831443211026, -84.77052832075908],
          [0, -84.77052832075908],
          [0, 84.89714695160266]]])

east_hemisphere = ee.Geometry.Polygon(
        [[[0, 84.89714695160266],
          [0, -84.77052832075908],
          [173.5310605678897, -84.77052832075908],
          [173.5310605678897, 84.89714695160266]]])

##### Define other implementation variables

In [116]:
# Earth engine username, used to import classified image into ee assets folder
USERNAME = 'wrfitch'
OUTPUT_DIR = USERNAME + "/out/"

# Define collections for each dataset to be used 
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
s5_CO = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CO")
s5_HCHO = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_HCHO")
s5_NO2 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")
s5_O3 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_O3")
s5_SO2 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_SO2")
s5_CH4 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4")

fs2 = ee.FeatureCollection("COPERNICUS/S2_SR")
fs5_CO = ee.FeatureCollection("COPERNICUS/S5P/OFFL/L3_CO")
fs5_HCHO = ee.FeatureCollection("COPERNICUS/S5P/OFFL/L3_HCHO")
fs5_NO2 = ee.FeatureCollection("COPERNICUS/S5P/OFFL/L3_NO2")
fs5_O3 = ee.FeatureCollection("COPERNICUS/S5P/OFFL/L3_O3")
fs5_SO2 = ee.FeatureCollection("COPERNICUS/S5P/OFFL/L3_SO2")
fs5_CH4 = ee.FeatureCollection("COPERNICUS/S5P/OFFL/L3_CH4")
#TODO import CO2 dataset

CO_band = 'CO_column_number_density'
HCHO_band = 'tropospheric_HCHO_column_number_density'
NO2_band = 'tropospheric_NO2_column_number_density'
O3_band = 'O3_column_number_density'
SO2_band = 'SO2_column_number_density'
CH4_band = 'CH4_column_volume_mixing_ratio_dry_air'

#define crs object so we can use the same coordinate reference system on everything. 
#crs = ee.Projection.

# Could the start and end dates be shifted or focused on one area, so emissions can be monitored across the seasons? 
# would that even be useful? 
start_date = '2020-01-01'
end_date = '2020-12-31'
vis_palette = ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']

drive_path = "/content/drive/MyDrive/"
export_dir = "img_export"

In [6]:
CO_fc = fs5_CO.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(CO_band)

HCHO_fc = fs5_HCHO.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(CO_band)

NO2_fc = fs5_NO2.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(CO_band)

O3_fc = fs5_O3.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(CO_band)

SO2_fc = fs5_SO2.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(CO_band)
  
CH4_fc = fs5_CH4.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(CO_band)

In [7]:
# Import datasets 
# TODO analyse whether these min/max values are valid, recalibrate for highest variance where necessary. Separate values
#      may be necessary for different samples - for example, the perfect calibration for the UK won't work on the world. 
# TODO analyse whether it makes sense to analyse these on a highly localised level
# Filterbounds doesn't even seem to work locally, so may as well remove it

# pre-filter to remove clouds - we can add them back in as data points from sentinel 5 if necessary
def maskS2clouds(image) :
  qa = image.select('QA60');

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bitmask = 1 << 10
  cirrus_bitmask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloud_bitmask).eq(0).And( \
         qa.bitwiseAnd(cirrus_bitmask).eq(0))

  return image.updateMask(mask).divide(10000)

# High-resolution satellite photograph 
s2_img = s2.filterDate(start_date, end_date) \
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
           .filterBounds(great_britain) \
           .map(maskS2clouds).median()
s2_id = s2_img.getMapId({'bands': ['B4', 'B3', 'B2'], \
                        'min': 0, \
                        'max': 0.3})

# Carbon monoxide
# Minmax scale is a bit off - recalibrate for Britain 
CO_img = s5_CO.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(CO_band).mean()
CO_id = CO_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0, \
    'max': 0.05})

# Formaldehyde
# Minmax scale is a bit off - recalibrate for Britain
HCHO_img = s5_HCHO.filterDate(start_date, end_date) \
                  .filterBounds(great_britain) \
                  .select(HCHO_band).mean()
HCHO_id = HCHO_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.0, \
    'max': 0.0003})

# Nitrogen Dioxide
NO2_img = s5_NO2.filterDate(start_date, end_date) \
                .filterBounds(great_britain) \
                .select(NO2_band).mean()
NO2_id = NO2_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.0, \
    'max': 0.0002})

# Ozone
O3_img = s5_O3.filterDate(start_date, end_date) \
              .filterBounds(great_britain) \
              .select(O3_band).mean()
O3_id = O3_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.12, \
    'max': 0.15})

# Sulphur Dioxide
SO2_img = s5_SO2.filterDate(start_date, end_date) \
                .filterBounds(great_britain) \
                .select(SO2_band).mean()
SO2_id = SO2_img.getMapId( \
    {'palette': vis_palette, \
    'min': 0.0, \
    'max': 0.0005})

# Methane
CH4_img = s5_CH4.filterDate(start_date, end_date) \
                .filterBounds(great_britain) \
                .select(CH4_band).mean()
CH4_id = CH4_img.getMapId( \
    {'palette': vis_palette, \
    'min': 1750, \
    'max': 1900})

In [8]:
# For easier iteration down the line. I know I'm not supposed to, but google can't tell me what to do, even if it's a good idea!
ghg_imgs = [CO_img, HCHO_img, NO2_img, O3_img, SO2_img, CH4_img]
ghg_ids = [CO_id, HCHO_id, NO2_id, O3_id, SO2_id, CH4_id]
ghg_fcs = [CO_fc, HCHO_fc, NO2_fc, O3_fc, SO2_fc, CH4_fc]

### Visualise Data

In [ ]:
# Visualise data on a Folium map 
# Attribution has to stay earthengine.google.com, since that's where these maps came from. 
map = folium.Map(
    location = [51.5, 0.1], 
    prefer_canvas = True)

layerOpacity = 0.5

folium.TileLayer(
    tiles = s2_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
    name = 'satellite photography median composite '
  ).add_to(map)

folium.TileLayer(
    tiles = CO_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
    name = 'Carbon Monoxide',
    opacity = layerOpacity
  ).add_to(map)

folium.TileLayer(
    tiles = HCHO_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
    name = 'Formaldehyde',
    opacity = layerOpacity
  ).add_to(map)

folium.TileLayer(
    tiles = NO2_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
    name = 'Nitrogen Dioxide',
    opacity = layerOpacity
  ).add_to(map)

folium.TileLayer(
    tiles = O3_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
    name = 'Ozone',
    opacity = layerOpacity
  ).add_to(map)

folium.TileLayer(
    tiles = SO2_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
    name = 'Sulphur Dioxide',
    opacity = layerOpacity
  ).add_to(map)

folium.TileLayer(
    tiles = CH4_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
    name = 'Methane',
    opacity = layerOpacity
  ).add_to(map)
  
map.add_child(folium.LayerControl())
map

### Export Data

Exports as .csv tables and GeoTIFF images. 

#### Define export methods

In [9]:
# All export methods export to the google drive defined above 
# TODO add checking for existing files in these methods.

def exportTable(table, scale, folder="no_export_folder", desc="no_desc"):
  ee.batch.Export.table.toDrive(
    collection = table,
    description = desc,
    folder = folder,
    fileFormat = "CSV"
  ).start()

# Export one table of the given image, at the scale and dimensions specified.
def exportTableFromImage(image, polygon, scale, folder="no_export_folder", desc="no_desc"):
  exportTable(sample(image, polygon, scale), scale, folder, desc)
  
# samples image into feature_collection. 
# TODO why is this called sample() ?
def sample(img, region, scale):
  return img.sampleRegions(
      collection = region,
      geometries = True,
      scale = scale
  )

# Export one GeoTIFF image of the given image, at the scale and dimension specified. 
# TODO reevaluate image export options - description needs coordinates
# maxPixels is just so it lets me export london at 10m/px. Dividing the dataset into 1km squares is the next step. 
def exportGeotiff(image, polygon, scale, folder="no_export_folder", desc="no_desc"):
  ee.batch.Export.image.toDrive(
    crs = 'EPSG:3857',
    description = desc,
    fileFormat = 'GeoTIFF',
    folder = folder,
    # formatOptions = image_export_options,
    image = image,
    maxPixels = 10e9,
    region = polygon,
    scale = scale
  ).start()

In [ ]:
pprint(ee.batch.Task.list())

#### Exporting CSVs

This method of getting the data is very very stupid, but also it does exactly what I need. 

In [10]:
# I cleaned up so much bad code there's almost nothing left, and that's still bad. 
def getDataInPolygon(polygon):
  dir = "dumb_idea_test"
  for ghg_img in ghg_imgs:
    exportTableFromImage(ghg_img, polygon, 1000, dir, ghg_img.getInfo().get('bands')[0].get('id'))

In [ ]:
# Only once this is completed can you move forward and get pictures from these spreadsheets. 
getDataInPolygon(london)
for ghg_img in ghg_imgs:
  exportTableFromImage(ghg_img, polygon, 1000, export_dir, ghg_img.getInfo().get('bands')[0].get('id'))

#### Getting Images From CSV Data

In [115]:
import csv 
import json
import time

#Gets a square kilometer with the given point object as the centroid. 
def getSqKmFromPoint(point):
  return point.buffer(500).bounds()

# Read in CSV from drive
# This method is bad, and I should feel bad 
def getImgsFromCsv(csv_path):
  with open(csv_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    concurrent_exports = 3
    exporting = []

    for row in csv_reader:
      if line_count == 0:
        print(f'Column names are {", ".join(row)}')
        line_count += 1
      else:
        # slightly less clumsy rate limiter (On^3 though...)
        # Block until export buffer is smaller than the maximum allowed concurrent exports. 
        # when the files stored in the export buffer are found in the filesystem, remove them.
        while len(exporting) >= concurrent_exports:
          print("waiting for current exports to finish, so as not to overload the system... ")
          for filename in exporting:
            # 7 layers of indentation?!?!
            if os.path.exists(filename): 
              exporting.remove(filename)
              time.sleep(5)

        coords = json.loads(row[2]).get("coordinates")
        poly = getSqKmFromPoint(ee.Geometry.Point(coords))
        name = str(coords[0]) + "_" + str(coords[1])
        tifname = name + ".tif"
        exporting.append(tifname)

        # skip files that have already been exported. 
        # TODO fix this mess
        if os.path.isfile(png_dir + "/" + name + ".png") or os.path.isfile(geotiff_dir + "/" + tifname): 
          continue
        
        print("exporting " + tifname)
        exportGeotiff(s2_img, poly, 10, export_dir + "/geotiff", name)

        line_count += 1

    print(f'Processed {line_count} lines.')

In [ ]:
getImgsFromCsv(drive_path + export_dir + "/SO2_column_number_density.csv")

# Data processing

### TODO
- Reorganise datasets for fast.ai retraining. 2-400 images was sufficient for object recog, maybe double that for top-down sat photos?


In [133]:
# removes old geoTIFF images or xml conversion artifacts from the given directory. 
def rmArtifact(artifact_path, rmTif = False, rmXml = False):
  # we only want to remove files, not directories! 
  if not os.path.isfile(artifact_path):
    return
  if not (rmTif or rmXml): 
    return

  split_path = os.path.splitext(artifact_path)
  if (split_path[1].lower() == ".tif" and rmTif) or \
    (split_path[1].lower() == ".xml" and rmXml): 
    print("removing " + artifact_path)
    os.remove(artifact_path)

def rmConversionArtifacts(path, rmTif = False, rmXml = False):
  # No point checking all these files if we're not going to do anything 
  if not (rmTif or rmXml): 
    return

  parent_path = os.path.join(drive_path, path)
  print(parent_path)

  for root, dirs, files in os.walk(parent_path, topdown=True):
    for name in files:
      fullpath = os.path.join(root, name)
      rmArtifact(fullpath, rmTif, rmXml)
  
# converts geotiff to png, using selected bands. There seems to be a limited range of functional bands, including only 3 
# being available for a non-transparent image. The bands also display in grayscale when displayed individually. 
# TODO refactor for readability - there are too many undefined "x-path" variables here that don't make a lot of sense 
def geotiffToPng(tif_path, rm_artifacts = False):
  #TODO remap to ARGB to get more defined brightness data
  options_list = [
    '-ot Byte',
    '-of PNG',
    '-b 4',
    '-b 3',
    '-b 2',
    '-scale'
  ]
  options_string = " ".join(options_list)
  
  parent_path = os.path.join(drive_path, tif_path)

  for root, dirs, files in os.walk(parent_path, topdown=False):
    for name in files:
      full_path = os.path.join(root, name)
      print(full_path)
      split_path = os.path.splitext(full_path)

      if split_path[1].lower() != ".tif": continue

      path = split_path[0]
      print(path)
      if os.path.isfile(path + ".png"):
        print("A png file already exists for %s" % fullpath)
        continue
      
      gdal.Translate(
        path + '.png',
        path + '.tif',
        options = options_string
      )
      if rm_artifacts: rmArtifact(full_path, True, True)

# Move files from src to dest if they have the correct extension
def moveFilesByExtension(src, dest, extension):
  parent_path = os.path.join(drive_path, src)
  print(parent_path)

  for root, dirs, files in os.walk(parent_path, topdown=True):
    for name in files:
      full_path = os.path.join(root, name)
      split_path = os.path.splitext(full_path)
      if split_path[1].lower() == extension:
        dest_path = full_path.replace(src, dest)
        os.rename(full_path, dest_path)

def deDupe(path):
  print("removing duplicates")
  # get rid of any dupe filenames 
  # get rid of anything with (1)  (or some other number) in its name 

In [ ]:
geotiffToPng(export_dir + "/geotiff", rm_artifacts = False)

In [134]:
moveFilesByExtension(export_dir + "/geotiff", export_dir + "/png", ".png")
rmConversionArtifacts(export_dir + "/geotiff", rmTif=False, rmXml=True)

/content/drive/MyDrive/img_export/geotiff
/content/drive/MyDrive/img_export/geotiff
removing /content/drive/MyDrive/img_export/geotiff/-0.7860258736045813_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.7770427207633861_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.7950090264457765_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.7590764150809957_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.7680595679221909_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.7411101093986052_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.7500932622398004_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.73212695655741_51.109648089980176.png.aux.xml
removing /content/drive/MyDrive/img_export/geotiff/-0.7231438037162148_51.109648089980176.png.aux.xml
